# TAREFA 5

Criar uma tabela com uma chave simples e alguns dados de exemplo. Cada valor de chave é um número
incremental e está associado a com valores que variam de 0 até 10

O que entregar: Imprimir os valores das 10 primeiras tuplas da tabela, ordenando por k.

In [2]:
pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 5.5 MB/s  0:00:00 eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


Iniciar conexão com o banco de dados

In [8]:
!ps aux | grep "bin/postgres -D"


!lsof -i :5433
!kill -9 56471

!rm -rf ./meu_postgres
!/usr/lib/postgresql/14/bin/initdb -D ./meu_postgres 

!/usr/lib/postgresql/14/bin/pg_ctl -D ./meu_postgres  -o "-p 5433 -h 127.0.0.1 -c unix_socket_directories='.'" -l logfile start


#-h 127.0.0.1 -c unix_socket_directories='.'
#!psql -h localhost -p5433 postgres

# Executa o comando SQL (entre aspas) no banco de dados "postgres" na porta 5433
!/usr/lib/postgresql/14/bin/psql -h localhost -p 5433 -c "CREATE USER michael WITH PASSWORD 'mwpv0301';" postgres

!/usr/lib/postgresql/14/bin/psql -h localhost -p 5433 -c "CREATE DATABASE notebook_bd OWNER michael;" postgres

!/usr/lib/postgresql/14/bin/psql -h localhost -p 5433 -c "GRANT ALL PRIVILEGES ON DATABASE notebook_bd TO michael;" postgres


postgres    1400  0.0  0.1 220968 30404 ?        Ss   10:16   0:00 /usr/lib/postgresql/14/bin/postgres -D /var/lib/postgresql/14/main -c config_file=/etc/postgresql/14/main/postgresql.conf
icomp       8365  0.0  0.0   9984  3528 pts/2    Ss+  10:25   0:00 /bin/bash -c ps aux | grep "bin/postgres -D"
icomp       8367  0.0  0.0   9244  2276 pts/2    S+   10:25   0:00 grep bin/postgres -D
/bin/bash: linha 1: kill: (56471) - Processo inexistente
The files belonging to this database system will be owned by user "icomp".
This user must also own the server process.

The database cluster will be initialized with locale "pt_BR.UTF-8".
The default database encoding has accordingly been set to "UTF8".
The default text search configuration will be set to "portuguese".

Data page checksums are disabled.

creating directory ./meu_postgres ... ok
creating subdirectories ... ok
selecting dynamic shared memory implementation ... posix
selecting default max_connections ... 100
selecting default shared_b

In [7]:
!/usr/lib/postgresql/14/bin/pg_ctl -D ./meu_postgres  -o "-p 5433" -l logfile start

waiting for server to start.... stopped waiting
pg_ctl: could not start server
Examine the log output.


In [9]:

import psycopg2

conn = psycopg2.connect(
    dbname='notebook_bd',
    user='michael',
    password='mwpv0301',
    host='localhost',
    port='5433'
)


cursor = conn.cursor()


Criar as tabelas juntamente com os dados para o banco de dados local

In [10]:
conn.rollback()

cursor.execute("DROP TABLE IF EXISTS t;")
cursor.execute("""CREATE TABLE t (k serial PRIMARY KEY, v integer);
""")
cursor.execute("""
INSERT INTO t(v)
SELECT trunc(random() * 10) FROM generate_series(1,100000);
""") 

conn.commit()

Mostrar as 10 primeiras linhas da tabela aleatória criada

In [11]:
conn.rollback()

cursor.execute("""
SELECT * FROM t
ORDER BY k
LIMIT 10;
""")

print(cursor.fetchall())

[(1, 2), (2, 2), (3, 9), (4, 4), (5, 9), (6, 5), (7, 4), (8, 3), (9, 5), (10, 4)]


# TAREFA 6

Verifique quantas páginas com blocos foram criadas para a tabela da Tarefa 5

In [12]:
cursor.execute("""
SELECT relname, relpages, reltuples FROM pg_class WHERE relname = 't';
""")

print(cursor.fetchall())

[('t', 0, -1.0)]


# TAREFA 7

Verifique quantos blocos foram efetivamente usados numa consulta

COMO EXECUTAR OS COMANDOS VIA NOTEBOOK?

\pset x on                    -- modo expandido no psql (formatação visual)


\SELECT pg_stat_reset();       -- reseta estatísticas (precisas de sudo eu acho)


\pset x off

In [13]:
conn.rollback()

cursor.execute("""
SELECT pg_sleep(1);               
""")

cursor.execute("""
SELECT * FROM pg_stats WHERE tablename = 't';               
""")
resultado = cursor.fetchall()


print(resultado)

[]


# TAREFA 17


In [63]:
import psycopg2
import random
import time
import threading
from psycopg2 import extensions, sql

# --- CONFIGURAÇÃO DO BANCO DE DADOS ---
# Preencha com suas credenciais reais
DB_HOST = "localhost"
DB_NAME = "notebook_bd" 
DB_USER = "michael"
DB_PASS = "mwpv0301"
DB_PORT = "5433"

# Parâmetros da Simulação
TOTAL_ASSENTOS = 10

In [54]:
class SistemaReservas:
    def __init__(self, isolation_level):
        """
        isolation_level: extensions.ISOLATION_LEVEL_READ_COMMITTED 
                         ou extensions.ISOLATION_LEVEL_SERIALIZABLE
        """
        self.isolation_level = isolation_level
        self.lock = threading.Lock() # Apenas para print thread-safe
        self.clientes_atendidos = 0 # Contador global para controle do experimento

    def get_connection(self):
        conn = psycopg2.connect(
            host=DB_HOST,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASS,
            port=DB_PORT
        )
        conn.set_isolation_level(self.isolation_level)
        return conn

    def setup_database(self):
        """Cria a tabela e popula com 200 assentos livres."""
        conn = psycopg2.connect(
            host=DB_HOST, database=DB_NAME, user=DB_USER, password=DB_PASS, port=DB_PORT
        )
        conn.set_isolation_level(extensions.ISOLATION_LEVEL_AUTOCOMMIT)
        cur = conn.cursor()
        
        # Recria tabela limpa
        cur.execute("DROP TABLE IF EXISTS Assentos")
        cur.execute("""
            CREATE TABLE Assentos (
                num_voo INTEGER PRIMARY KEY,
                disp BOOLEAN NOT NULL DEFAULT TRUE
            )
        """)
        
        # Popula 200 assentos
        values = [(i, True) for i in range(1, TOTAL_ASSENTOS + 1)]
        args_str = ','.join(cur.mogrify("(%s, %s)", x).decode('utf-8') for x in values)
        cur.execute("INSERT INTO Assentos (num_voo, disp) VALUES " + args_str)
        
        print(f"Banco resetado: {TOTAL_ASSENTOS} assentos disponíveis.")
        cur.close()
        conn.close()

    def get_assentos_disponiveis(self, cur):
        """Passo 1: Recupera lista de assentos disponíveis."""
        # Nota: Em Read Committed, isso é apenas um snapshot do momento do SELECT.
        # Em Serializable, isso pode afetar o conjunto de leitura.
        cur.execute("SELECT num_voo FROM Assentos WHERE disp = TRUE")
        return [row[0] for row in cur.fetchall()]

    def realizar_reserva_versao_A(self, thread_id):
        """
        Versão A: Transação única englobando Passo 1, 2 e 3.
        O sleep(1) ocorre DENTRO da transação aberta.
        """
        conn = self.get_connection()
        try:
            cur = conn.cursor()
            
            # --- INÍCIO TRANSAÇÃO ---
            # (Em psycopg2, a transação inicia no primeiro comando se autocommit=False)
            
            # Passo 1: Recupera assentos
            assentos = self.get_assentos_disponiveis(cur)
            
            if not assentos:
                conn.rollback()
                return False # Sem assentos, encerra tentativa

            # Passo 2: Escolha do cliente (1 segundo de delay)
            assento_escolhido = random.choice(assentos)
            time.sleep(1) 

            # Passo 3: Atualiza
            # Importante: Verificamos 'disp=TRUE' novamente no UPDATE para garantir atomicidade 
            # lógica, embora a trava de linha (row lock) aconteça aqui.
            cur.execute(
                "UPDATE Assentos SET disp = FALSE WHERE num_voo = %s AND disp = TRUE",
                (assento_escolhido,)
            )
            
            if cur.rowcount > 0:
                conn.commit()
                return True # Sucesso
            else:
                # Alguém pegou o assento enquanto eu dormia
                conn.rollback()
                return False
                
        except psycopg2.extensions.TransactionRollbackError:
            # Captura erros de serialização (comuns no nível Serializable)
            conn.rollback()
            return False
        except Exception as e:
            print(f"Erro Thread {thread_id}: {e}")
            conn.rollback()
            return False
        finally:
            conn.close()

    def realizar_reserva_versao_B(self, thread_id):
        """
        Versão B: Duas transações.
        Tx1: Leitura.
        [Sem Tx]: Sleep(1).
        Tx2: Escrita.
        """
        # --- Transação 1: Leitura ---
        conn1 = self.get_connection()
        try:
            cur1 = conn1.cursor()
            assentos = self.get_assentos_disponiveis(cur1)
            conn1.commit() # Fim da Tx1
        except Exception as e:
            conn1.rollback()
            conn1.close()
            return False
        finally:
            conn1.close()

        if not assentos:
            return False

        # --- Passo 2: Fora de transação ---
        assento_escolhido = random.choice(assentos)
        time.sleep(1)

        # --- Transação 2: Escrita ---
        conn2 = self.get_connection()
        try:
            cur2 = conn2.cursor()
            # Tenta reservar. Se outro agente reservou no intervalo, rowcount será 0
            cur2.execute(
                "UPDATE Assentos SET disp = FALSE WHERE num_voo = %s AND disp = TRUE",
                (assento_escolhido,)
            )
            
            if cur2.rowcount > 0:
                conn2.commit()
                return True
            else:
                conn2.rollback()
                return False
        except Exception as e:
            # print(f"Erro T2 Thread {thread_id}: {e}")
            conn2.rollback()
            return False
        finally:
            conn2.close()

    def worker(self, thread_id, versao):
        """Função executada por cada thread (Agente)."""
        while True:
            # Verificação simples de parada: se já vendemos tudo
            # Nota: Em um cenário real de alta concorrência, consultar o DB seria melhor,
            # mas para simulação usamos um contador ou consultamos o banco se falhar muito.
            
            conn_check = self.get_connection()
            cur_check = conn_check.cursor()
            cur_check.execute("SELECT COUNT(*) FROM Assentos WHERE disp = FALSE")
            vendidos = cur_check.fetchone()[0]
            conn_check.close()
            
            if vendidos >= TOTAL_ASSENTOS:
                break

            # Tenta realizar reserva
            sucesso = False
            if versao == 'a':
                sucesso = self.realizar_reserva_versao_A(thread_id)
            else:
                sucesso = self.realizar_reserva_versao_B(thread_id)
            
            if sucesso:
                # Opcional: Logar progresso
                pass

In [58]:
# --- CONFIGURAÇÃO DA EXECUÇÃO ---
# Escolha a versão: 'a' ou 'b'
VERSAO = 'a' 

# Escolha o nível: extensions.ISOLATION_LEVEL_READ_COMMITTED ou ISOLATION_LEVEL_SERIALIZABLE
NIVEL_ISOLAMENTO = extensions.ISOLATION_LEVEL_READ_COMMITTED

# Quantidade de agentes (threads): 1, 2, 4, 6, 8, 10
NUM_AGENTES = 4

# --- EXECUÇÃO ---
print(f"--- Iniciando Experimento: Versão {VERSAO.upper()} | Agentes: {NUM_AGENTES} ---")

sistema = SistemaReservas(NIVEL_ISOLAMENTO)
sistema.setup_database() # Reseta a tabela para true

start_time = time.time()

threads = []
for i in range(NUM_AGENTES):
    t = threading.Thread(target=sistema.worker, args=(i, VERSAO))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

end_time = time.time()
tempo_total = end_time - start_time

print(f"\nExperimento Concluído!")
print(f"Tempo Total: {tempo_total:.2f} segundos")

# Validação final
conn = sistema.get_connection()
cur = conn.cursor()
cur.execute("SELECT COUNT(*) FROM Assentos WHERE disp = FALSE")
total_vendidos = cur.fetchone()[0]
print(f"Total de assentos reservados no banco: {total_vendidos}/{TOTAL_ASSENTOS}")
conn.close()

--- Iniciando Experimento: Versão A | Agentes: 4 ---
Banco resetado: 200 assentos disponíveis.


KeyboardInterrupt: 

In [62]:
# --- EXECUÇÃO ---
def executar_experimento(NUM_AGENTES, VERSAO, NIVEL_ISOLAMENTO):
    print(f"--- Iniciando Experimento: Versão {VERSAO.upper()} | Agentes: {NUM_AGENTES} ---")

    sistema = SistemaReservas(NIVEL_ISOLAMENTO)
    sistema.setup_database() # Reseta a tabela para true

    start_time = time.time()

    threads = []
    for i in range(NUM_AGENTES):
        t = threading.Thread(target=sistema.worker, args=(i, VERSAO))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

    end_time = time.time()
    tempo_total = end_time - start_time

    print(f"\nExperimento Concluído!")
    print(f"Tempo Total: {tempo_total:.2f} segundos")

    # Validação final
    conn = sistema.get_connection()
    cur = conn.cursor()
    cur.execute("SELECT COUNT(*) FROM Assentos WHERE disp = FALSE")
    total_vendidos = cur.fetchone()[0]
    print(f"Total de assentos reservados no banco: {total_vendidos}/{TOTAL_ASSENTOS}")
    conn.close()

In [64]:
agentes_list = [1, 2, 4, 6, 8, 10]
versoes = ['a', 'b']
niveis_isolamento = [
    extensions.ISOLATION_LEVEL_READ_COMMITTED,
    extensions.ISOLATION_LEVEL_SERIALIZABLE
]

for versao in versoes:
    for nivel in niveis_isolamento:
        for num_agentes in agentes_list:
            executar_experimento(num_agentes, versao, nivel)
            print("\n" + "="*50 + "\n")

--- Iniciando Experimento: Versão A | Agentes: 1 ---
Banco resetado: 10 assentos disponíveis.

Experimento Concluído!
Tempo Total: 10.34 segundos
Total de assentos reservados no banco: 10/10


--- Iniciando Experimento: Versão A | Agentes: 2 ---
Banco resetado: 10 assentos disponíveis.

Experimento Concluído!
Tempo Total: 7.24 segundos
Total de assentos reservados no banco: 10/10


--- Iniciando Experimento: Versão A | Agentes: 4 ---
Banco resetado: 10 assentos disponíveis.

Experimento Concluído!
Tempo Total: 4.15 segundos
Total de assentos reservados no banco: 10/10


--- Iniciando Experimento: Versão A | Agentes: 6 ---
Banco resetado: 10 assentos disponíveis.

Experimento Concluído!
Tempo Total: 2.09 segundos
Total de assentos reservados no banco: 10/10


--- Iniciando Experimento: Versão A | Agentes: 8 ---
Banco resetado: 10 assentos disponíveis.

Experimento Concluído!
Tempo Total: 3.14 segundos
Total de assentos reservados no banco: 10/10


--- Iniciando Experimento: Versão A | A